# Training Hold2Vec Embeddings

In [1]:
import os, sys, time
sys.path.append('..')

from data_loading import *
from embeddings.route_embeddings import *
from embeddings import hold2vec

2023-04-27 22:32:03.536980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
window_size = 8
embedding_size = 100
objective='skip-gram'   # 'skip-gram' or 'cbow'
epochs = 20


export_path = '../data/embeddings/'
file_name = f'hold2vec_{objective}_embedding{embedding_size}_window{window_size}_epochs{epochs}.npy'

assert os.path.exists(export_path)
print(f'Output file is {export_path + file_name}')

Output file is ../data/embeddings/hold2vec_skip-gram_embedding100_window8_epochs20.npy


In [3]:
!ls

0. Data Exploration.ipynb             2. Training Hold2Vec Embeddings.ipynb
1. Preprocessing.ipynb                3. MLP Classifier.ipynb
2. Building Embeddings.ipynb


In [4]:
df = load_dataframe('../data/2017.json')
df.head()

,Grade,UserRating,Moves
0,6A+,1,"[H5, E7, F8, D10, E13, C14, C16, B18, E18, F9,..."
1,6A+,2,"[H5, E7, D8, D10, E13, C14, C16, B18, E18]"
2,6B+,2,"[A4, B4, C7, D9, F12, D15, F18]"
3,6A+,2,"[K4, J4, I7, H9, I10, F12, F13, E6, B14, C16, ..."
4,6B+,2,"[G8, D9, F12, C13, F15, D17, F18, E6, G4, F4]"


In [5]:
bag_of_holds_embeddings_1d = bag_of_holds_1d(df.Moves)
print(bag_of_holds_embeddings_1d.shape)

(18865, 198)


In [6]:
h2v = hold2vec.hold2vec()
h2v.initialize(N=embedding_size, window_size=window_size)

2023-04-27 22:32:07.490754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
if os.path.exists(export_path + file_name):
    print(f'File {export_path + file_name} already exists, abort training.')
else:    
    # Actual Training:
    
    start_time = time.time()
    print(f'Computing embeddings of size {embedding_size} with context window size {window_size}')
    
    h2v.build_training_data(bag_of_holds_embeddings_1d, objective=objective) # build cbow or skip-gram training data set
    h2v.build_model() # creates the keras.model
    h2v.train(epochs=epochs)  
    h2v.save_hold_matrix(filename = export_path + file_name) # write embeddings to file
    hold_embeddings = h2v.get_hold_matrix().T
    
    time_delta = time.time() - start_time
    print(f'all done in {time_delta // 60:.0f} minutes {time_delta % 60:.2f} seconds')
    print(hold_embeddings.shape)

File ../data/embeddings/hold2vec_skip-gram_embedding100_window8_epochs20.npy already exists, abort training.
